### Embeddings Demo

You can use this demo for embeddings

In [1]:
import numpy as np
import openai
import os
import pandas as pd
import re
import requests
import sys
import tiktoken

from dotenv import load_dotenv
from num2words import num2words
from openai.embeddings_utils import get_embedding, cosine_similarity

In [3]:
# Get Configuration Settings
load_dotenv()

# Region WE/EUS/CE
region = 'WE'

endpoint = f'AOAI_ENDPOINT_{region.upper()}'
key = f'AOAI_KEY_{region.upper()}'

In [12]:
# Set OpenAI Configuration
openai.api_type = "azure"
openai.api_base = os.getenv(endpoint)
openai.api_key = os.getenv(key)
openai.api_version = "2022-12-01"
deployment_name = "text-embedding-ada-002-2"

In [ ]:
# Check which models are currently deployed
url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 
r = requests.get(url, headers={"api-key": os.getenv(key)})
print(r.text)

In [4]:
df = pd.read_csv('bill_sum_data.csv') # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df.head()

,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278


In [5]:
df_bills = df[['text', 'summary', 'title']]
df_bills

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...
6,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...
7,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...
8,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013
9,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993


In [6]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

In [7]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

20

In [11]:
df_bills.head()

,text,summary,title,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038


In [9]:
df_bills.text[0]

"SECTION 1. SHORT TITLE. This Act may be cited as the ``National Science Education Tax Incentive for Businesses Act of 2007''. SEC. 2. CREDITS FOR CERTAIN CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY SCHOOL LEVEL. (a) In General.--Subpart D of part IV of subchapter A of chapter 1 of the Internal Revenue Code of 1986 (relating to business related credits) is amended by adding at the end the following new section: ``SEC. 45O. CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY SCHOOL LEVEL. ``(a) In General.--For purposes of section 38, the elementary and secondary science, technology, engineering, and mathematics (STEM) contributions credit determined under this section for the taxable year is an amount equal to 100 percent of the qualified STEM contributions of the taxpayer for such taxable year. ``(b) Qualified STEM Contributions.--For purposes of th

In [8]:
sample_encode = tokenizer.encode(df_bills.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'SECTION',
 b' ',
 b'1',
 b'.',
 b' SHORT',
 b' TITLE',
 b'.',
 b' This',
 b' Act',
 b' may',
 b' be',
 b' cited',
 b' as',
 b' the',
 b' ``',
 b'National',
 b' Science',
 b' Education',
 b' Tax',
 b' In',
 b'cent',
 b'ive',
 b' for',
 b' Businesses',
 b' Act',
 b' of',
 b' ',
 b'200',
 b'7',
 b"''.",
 b' SEC',
 b'.',
 b' ',
 b'2',
 b'.',
 b' C',
 b'RED',
 b'ITS',
 b' FOR',
 b' CERT',
 b'AIN',
 b' CONTRIBUT',
 b'IONS',
 b' BEN',
 b'EF',
 b'IT',
 b'ING',
 b' SC',
 b'IENCE',
 b',',
 b' TECHNO',
 b'LOGY',
 b',',
 b' ENGINE',
 b'ERING',
 b',',
 b' AND',
 b' MAT',
 b'HE',
 b'MAT',
 b'ICS',
 b' EDUC',
 b'ATION',
 b' AT',
 b' THE',
 b' ELEMENT',
 b'ARY',
 b' AND',
 b' SECOND',
 b'ARY',
 b' SCHOOL',
 b' LEVEL',
 b'.',
 b' (',
 b'a',
 b')',
 b' In',
 b' General',
 b'.--',
 b'Sub',
 b'part',
 b' D',
 b' of',
 b' part',
 b' IV',
 b' of',
 b' sub',
 b'chapter',
 b' A',
 b' of',
 b' chapter',
 b' ',
 b'1',
 b' of',
 b' the',
 b' Internal',
 b' Revenue',
 b' Code',
 b' of',
 b' ',
 b'198',
 b'6',


In [10]:
len(decode)

1466

In [13]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = deployment_name)) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [14]:
df_bills.head()

,text,summary,title,n_tokens,ada_v2
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466,"[-0.022678455337882042, -0.00799493957310915, ..."
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[-0.041596777737140656, -0.009042778052389622,..."
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937,"[-0.0420723482966423, -0.0023871997836977243, ..."
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670,"[-0.013265267945826054, -0.007186719682067633,..."
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038,"[-0.03917262703180313, -0.025464843958616257, ..."


In [18]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine = deployment_name 
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

In [19]:
res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)

,text,summary,title,n_tokens,ada_v2,similarities
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947,"[-0.018460797145962715, -0.024794351309537888,...",0.819775
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[-0.041596777737140656, -0.009042778052389622,...",0.734921
11,SECTION 1. SHORT TITLE. This Act may be cited ...,Wall Street Compensation Reform Act of 2010 - ...,A bill to amend the Internal Revenue Code of 1...,2331,"[-0.047416992485523224, -0.007824326865375042,...",0.734551
19,SECTION 1. SHORT TITLE. This Act may be cited ...,Strengthening the Health Care Safety Net Act o...,To amend title XIX of the Social Security Act ...,2678,"[-0.017817417159676552, -0.007099970709532499,...",0.733241


In [20]:
res["summary"][9]

"Taxpayer's Right to View Act of 1993 - Amends the Communications Act of 1934 to prohibit a cable operator from assessing separate charges for any video programming of a sporting, theatrical, or other entertainment event if that event is performed at a facility constructed, renovated, or maintained with tax revenues or by an organization that receives public financial support. Authorizes the Federal Communications Commission and local franchising authorities to make determinations concerning the applicability of such prohibition. Sets forth conditions under which a facility is considered to have been constructed, maintained, or renovated with tax revenues. Considers events performed by nonprofit or public organizations that receive tax subsidies to be subject to this Act if the event is sponsored by, or includes the participation of a team that is part of, a tax exempt organization."